<a href="https://colab.research.google.com/github/neeraj-singh04/Deep-Learning/blob/master/OptimizerCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install keras-tuner

In [0]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [0]:
print(tf.__version__)

2.1.0-rc2


In [0]:
fashion_mnist = keras.datasets.fashion_mnist

In [0]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

In [0]:
train_images=train_images/255.0
test_images=test_images/255.0


In [0]:
train_images[0].shape

(28, 28)

In [0]:
train_images=train_images.reshape(len(train_images),28,28,1)

In [0]:
test_images=test_images.reshape(len(test_images),28,28,1)

In [0]:
def build_model(hp):
  model = keras.Sequential([
                            keras.layers.Conv2D(
                                filters = hp.Int('conv_1_filter',min_value=32,max_value=128,step=16),
                                kernel_size=hp.Choice('conv_1_kernel',values=[3,5]),
                                activation='relu',
                                input_shape=(28,28,1)

                            ),
                            keras.layers.Conv2D(
                                filters=hp.Int('conv_2_filter',min_value=32,max_value=64,step=16),
                                kernel_size=hp.Choice('conv_2_kernel',values=[3,5]),
                                activation='relu'
                            ),
                            keras.layers.Flatten(),
                            keras.layers.Dense(
                                units=hp.Int('dense_1_units',min_value=32,max_value=128,step=16),
                                activation='relu'
                            ),
                            keras.layers.Dense(10,activation='softmax')
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])),
                loss= 'sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model

In [0]:
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters


In [0]:
tuner_search = RandomSearch(build_model,
                            objective='val_accuracy',
                            max_trials=5,
                            directory='output',
                            project_name='MNIST Fashion')

In [0]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Train on 54000 samples, validate on 6000 samples
Epoch 1/3
54000/54000 [==============================] - 14s 252us/sample - loss: 0.4968 - accuracy: 0.8230 - val_loss: 0.4128 - val_accuracy: 0.8442
Epoch 2/3
54000/54000 [==============================] - 7s 139us/sample - loss: 0.3825 - accuracy: 0.8588 - val_loss: 0.3726 - val_accuracy: 0.8598
Epoch 3/3
54000/54000 [==============================] - 8s 140us/sample - loss: 0.3571 - accuracy: 0.8673 - val_loss: 0.4126 - val_accuracy: 0.8570


Train on 54000 samples, validate on 6000 samples
Epoch 1/3
54000/54000 [==============================] - 8s 148us/sample - loss: 0.5316 - accuracy: 0.8193 - val_loss: 0.4151 - val_accuracy: 0.8477
Epoch 2/3
54000/54000 [==============================] - 8s 140us/sample - loss: 0.3970 - accuracy: 0.8544 - val_loss: 0.3851 - val_accuracy: 0.8583
Epoch 3/3
54000/54000 [==============================] - 8s 142us/sample - loss: 0.3712 - accuracy: 0.8640 - val_loss: 0.3946 - val_accuracy: 0.8610


Train on 54000 samples, validate on 6000 samples
Epoch 1/3
54000/54000 [==============================] - 8s 142us/sample - loss: 2.3202 - accuracy: 0.0966 - val_loss: 2.3043 - val_accuracy: 0.0973
Epoch 2/3
54000/54000 [==============================] - 7s 134us/sample - loss: 2.3038 - accuracy: 0.1013 - val_loss: 2.3028 - val_accuracy: 0.1003
Epoch 3/3
54000/54000 [==============================] - 7s 133us/sample - loss: 2.3040 - accuracy: 0.0996 - val_loss: 2.3043 - val_accuracy: 0.0925


Train on 54000 samples, validate on 6000 samples
Epoch 1/3
54000/54000 [==============================] - 8s 155us/sample - loss: 0.5288 - accuracy: 0.8085 - val_loss: 0.4067 - val_accuracy: 0.8510
Epoch 2/3
54000/54000 [==============================] - 8s 150us/sample - loss: 0.4018 - accuracy: 0.8514 - val_loss: 0.4605 - val_accuracy: 0.8420
Epoch 3/3
54000/54000 [==============================] - 8s 155us/sample - loss: 0.3725 - accuracy: 0.8605 - val_loss: 0.3832 - val_accuracy: 0.8612


Train on 54000 samples, validate on 6000 samples
Epoch 1/3
54000/54000 [==============================] - 7s 122us/sample - loss: 0.4033 - accuracy: 0.8545 - val_loss: 0.2983 - val_accuracy: 0.8915
Epoch 2/3
54000/54000 [==============================] - 6s 113us/sample - loss: 0.2630 - accuracy: 0.9037 - val_loss: 0.2993 - val_accuracy: 0.8883
Epoch 3/3
54000/54000 [==============================] - 6s 114us/sample - loss: 0.2086 - accuracy: 0.9228 - val_loss: 0.2445 - val_accuracy: 0.9128


INFO:tensorflow:Oracle triggered exit


In [0]:
model = tuner_search.get_best_models(num_models=1)[0]

In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 32)        832       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 32)        9248      
_________________________________________________________________
flatten (Flatten)            (None, 15488)             0         
_________________________________________________________________
dense (Dense)                (None, 96)                1486944   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                970       
Total params: 1,497,994
Trainable params: 1,497,994
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.fit(train_images,train_labels,epochs=10,validation_split=0.1,initial_epoch=3)

Train on 54000 samples, validate on 6000 samples
Epoch 4/10
54000/54000 [==============================] - 7s 121us/sample - loss: 0.1614 - accuracy: 0.9395 - val_loss: 0.2486 - val_accuracy: 0.9165
Epoch 5/10
54000/54000 [==============================] - 6s 116us/sample - loss: 0.1222 - accuracy: 0.9540 - val_loss: 0.2810 - val_accuracy: 0.9070
Epoch 6/10
54000/54000 [==============================] - 6s 116us/sample - loss: 0.0935 - accuracy: 0.9650 - val_loss: 0.2891 - val_accuracy: 0.9112
Epoch 7/10
54000/54000 [==============================] - 6s 114us/sample - loss: 0.0700 - accuracy: 0.9741 - val_loss: 0.3425 - val_accuracy: 0.9122
Epoch 8/10
54000/54000 [==============================] - 6s 112us/sample - loss: 0.0530 - accuracy: 0.9801 - val_loss: 0.3388 - val_accuracy: 0.9127
Epoch 9/10
54000/54000 [==============================] - 6s 113us/sample - loss: 0.0448 - accuracy: 0.9839 - val_loss: 0.4389 - val_accuracy: 0.9100
Epoch 10/10
54000/54000 [==========================